In [1]:
import pandas as pd
from graph_tool import Graph
from netpixi.data.gt import *
import netpixi
from graph_tool import centrality
import seaborn as sns
import matplotlib.pyplot as plt
from graph_tool import clustering
import cpnet
import statsmodels.formula.api as smf
from graph_tool import spectral
from matplotlib import rcParams
import numpy as np
from math import sqrt
import csv

# Importando o Dataset

In [2]:
!pip install opendatasets

     |████████████████████████████████| 97 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 149 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 10.3 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=d4cf1190865cdd035acc6fe34b9e29cd0dfb784a62b6c88d31fffd523f82e38d
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import opendatasets as od

In [4]:
od.download("https://www.kaggle.com/stackoverflow/stacksample")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: beatrizmuniz
Your Kaggle Key: ········


100%|██████████| 1.11G/1.11G [03:19<00:00, 5.97MB/s]


# Lendo os Dados

In [11]:
tags = pd.read_csv("stacksample/Tags.csv")
quests = pd.read_csv("stacksample/Questions.csv", encoding='ISO-8859-1', engine='python')
answ = pd.read_csv("stacksample/Answers.csv", encoding='ISO-8859-1', engine='python')

In [13]:
#filtrando perguntas apenas para aquelas de 2016 
quests = quests[(quests['CreationDate'] >= '2016-10-00T00:00:00Z')]

In [14]:
#numero total de perguntas em 2016
quests.shape[0]

14590

In [15]:
#filtrando as tags para pegar apenas as dos posts de 2016
tags = tags[tags['Id'].isin(quests['Id'])]

In [16]:
#quantidade de tags no total (não únicas; cada fileira representa uma tag associada a uma pergunta;uma tag pode ser 
#associada a mais de uma pergunta)
tags.shape[0]

43377

In [17]:
#filtrando as tags para manter apenas as perguntas que possuem duas ou mais tags associadas
tags = tags.groupby('Id').filter(lambda x: len(x) > 2)

In [18]:
#quantidade de tags depois de filtrar
tags.shape[0]

34010

In [19]:
#filtrando as perguntas para ter apenas as com 2 ou mais tags
quests = quests[quests['Id'].isin(tags['Id'])]

In [20]:
#quantidade de perguntas depois de filtrar
quests.shape[0]

8977

# Fazendo o grafo

In [21]:
#setup do grafo
g = Graph(directed=False) # não dirigido
g.vp['name'] = g.new_vp('object')
g.ep['counter'] = g.new_ep('object')

In [30]:
# adiciona vertice de uma tag ao grafo
def addTag(tag, list_tags):
    g.add_vertex()
    list_tags.append(tag)
    v_tag = g.vertex(list_tags.index(tag))
    g.vp['name'][v_tag] = str(tag)

In [23]:
#adiciona aresta ao grafo quando duas tags estão presentes na mesma pergunta
def addConnec(tag1, tag2):
    index1 = list_tags.index(tag1)
    index2 = list_tags.index(tag2)
    
    if g.edge(index1, index2) == None:
        g.add_edge(index1, index2)
        e=g.edge(index1, index2)
        g.ep['counter'][e] = 1
    else:
        e=g.edge(index1, index2)
        g.ep['counter'][e] = g.ep['counter'][e] + 1

In [31]:
list_tags = []
for tagIndex in range(tags.shape[0] - 2):
    tag = tags.iloc[tagIndex]['Tag']
    questId = tags.iloc[tagIndex]['Id']
    
    if tag not in list_tags:
        addTag(tag, list_tags)
    
    sameQuest = True
    i=1
    
    print(tagIndex)
    while sameQuest:
        nextTag = tags.iloc[tagIndex + i]['Tag']
        nextId = tags.iloc[tagIndex + i]['Id']

        if questId == nextId:
            if nextTag not in list_tags:
                addTag(nextTag, list_tags)

            addConnec(tag, nextTag)
        else:
            sameQuest = False
            print("prox vertice")
        
        if ((tagIndex + i) < (tags.shape[0] - 1)):
            i += 1
        else:
            sameQuest = False

0
prox vertice
1
prox vertice
2
prox vertice
3
prox vertice
4
prox vertice
5
prox vertice
6
prox vertice
7
prox vertice
8
prox vertice
9
prox vertice
10
prox vertice
11
prox vertice
12
prox vertice
13
prox vertice
14
prox vertice
15
prox vertice
16
prox vertice
17
prox vertice
18
prox vertice
19
prox vertice
20
prox vertice
21
prox vertice
22
prox vertice
23
prox vertice
24
prox vertice
25
prox vertice
26
prox vertice
27
prox vertice
28
prox vertice
29
prox vertice
30
prox vertice
31
prox vertice
32
prox vertice
33
prox vertice
34
prox vertice
35
prox vertice
36
prox vertice
37
prox vertice
38
prox vertice
39
prox vertice
40
prox vertice
41
prox vertice
42
prox vertice
43
prox vertice
44
prox vertice
45
prox vertice
46
prox vertice
47
prox vertice
48
prox vertice
49
prox vertice
50
prox vertice
51
prox vertice
52
prox vertice
53
prox vertice
54
prox vertice
55
prox vertice
56
prox vertice
57
prox vertice
58
prox vertice
59
prox vertice
60
prox vertice
61
prox vertice
62
prox vertice
63

prox vertice
531
prox vertice
532
prox vertice
533
prox vertice
534
prox vertice
535
prox vertice
536
prox vertice
537
prox vertice
538
prox vertice
539
prox vertice
540
prox vertice
541
prox vertice
542
prox vertice
543
prox vertice
544
prox vertice
545
prox vertice
546
prox vertice
547
prox vertice
548
prox vertice
549
prox vertice
550
prox vertice
551
prox vertice
552
prox vertice
553
prox vertice
554
prox vertice
555
prox vertice
556
prox vertice
557
prox vertice
558
prox vertice
559
prox vertice
560
prox vertice
561
prox vertice
562
prox vertice
563
prox vertice
564
prox vertice
565
prox vertice
566
prox vertice
567
prox vertice
568
prox vertice
569
prox vertice
570
prox vertice
571
prox vertice
572
prox vertice
573
prox vertice
574
prox vertice
575
prox vertice
576
prox vertice
577
prox vertice
578
prox vertice
579
prox vertice
580
prox vertice
581
prox vertice
582
prox vertice
583
prox vertice
584
prox vertice
585
prox vertice
586
prox vertice
587
prox vertice
588
prox vertice
5

1029
prox vertice
1030
prox vertice
1031
prox vertice
1032
prox vertice
1033
prox vertice
1034
prox vertice
1035
prox vertice
1036
prox vertice
1037
prox vertice
1038
prox vertice
1039
prox vertice
1040
prox vertice
1041
prox vertice
1042
prox vertice
1043
prox vertice
1044
prox vertice
1045
prox vertice
1046
prox vertice
1047
prox vertice
1048
prox vertice
1049
prox vertice
1050
prox vertice
1051
prox vertice
1052
prox vertice
1053
prox vertice
1054
prox vertice
1055
prox vertice
1056
prox vertice
1057
prox vertice
1058
prox vertice
1059
prox vertice
1060
prox vertice
1061
prox vertice
1062
prox vertice
1063
prox vertice
1064
prox vertice
1065
prox vertice
1066
prox vertice
1067
prox vertice
1068
prox vertice
1069
prox vertice
1070
prox vertice
1071
prox vertice
1072
prox vertice
1073
prox vertice
1074
prox vertice
1075
prox vertice
1076
prox vertice
1077
prox vertice
1078
prox vertice
1079
prox vertice
1080
prox vertice
1081
prox vertice
1082
prox vertice
1083
prox vertice
1084
prox 

prox vertice
1658
prox vertice
1659
prox vertice
1660
prox vertice
1661
prox vertice
1662
prox vertice
1663
prox vertice
1664
prox vertice
1665
prox vertice
1666
prox vertice
1667
prox vertice
1668
prox vertice
1669
prox vertice
1670
prox vertice
1671
prox vertice
1672
prox vertice
1673
prox vertice
1674
prox vertice
1675
prox vertice
1676
prox vertice
1677
prox vertice
1678
prox vertice
1679
prox vertice
1680
prox vertice
1681
prox vertice
1682
prox vertice
1683
prox vertice
1684
prox vertice
1685
prox vertice
1686
prox vertice
1687
prox vertice
1688
prox vertice
1689
prox vertice
1690
prox vertice
1691
prox vertice
1692
prox vertice
1693
prox vertice
1694
prox vertice
1695
prox vertice
1696
prox vertice
1697
prox vertice
1698
prox vertice
1699
prox vertice
1700
prox vertice
1701
prox vertice
1702
prox vertice
1703
prox vertice
1704
prox vertice
1705
prox vertice
1706
prox vertice
1707
prox vertice
1708
prox vertice
1709
prox vertice
1710
prox vertice
1711
prox vertice
1712
prox verti

prox vertice
2277
prox vertice
2278
prox vertice
2279
prox vertice
2280
prox vertice
2281
prox vertice
2282
prox vertice
2283
prox vertice
2284
prox vertice
2285
prox vertice
2286
prox vertice
2287
prox vertice
2288
prox vertice
2289
prox vertice
2290
prox vertice
2291
prox vertice
2292
prox vertice
2293
prox vertice
2294
prox vertice
2295
prox vertice
2296
prox vertice
2297
prox vertice
2298
prox vertice
2299
prox vertice
2300
prox vertice
2301
prox vertice
2302
prox vertice
2303
prox vertice
2304
prox vertice
2305
prox vertice
2306
prox vertice
2307
prox vertice
2308
prox vertice
2309
prox vertice
2310
prox vertice
2311
prox vertice
2312
prox vertice
2313
prox vertice
2314
prox vertice
2315
prox vertice
2316
prox vertice
2317
prox vertice
2318
prox vertice
2319
prox vertice
2320
prox vertice
2321
prox vertice
2322
prox vertice
2323
prox vertice
2324
prox vertice
2325
prox vertice
2326
prox vertice
2327
prox vertice
2328
prox vertice
2329
prox vertice
2330
prox vertice
2331
prox verti

...

prox vertice
31779
prox vertice
31780
prox vertice
31781
prox vertice
31782
prox vertice
31783
prox vertice
31784
prox vertice
31785
prox vertice
31786
prox vertice
31787
prox vertice
31788
prox vertice
31789
prox vertice
31790
prox vertice
31791
prox vertice
31792
prox vertice
31793
prox vertice
31794
prox vertice
31795
prox vertice
31796
prox vertice
31797
prox vertice
31798
prox vertice
31799
prox vertice
31800
prox vertice
31801
prox vertice
31802
prox vertice
31803
prox vertice
31804
prox vertice
31805
prox vertice
31806
prox vertice
31807
prox vertice
31808
prox vertice
31809
prox vertice
31810
prox vertice
31811
prox vertice
31812
prox vertice
31813
prox vertice
31814
prox vertice
31815
prox vertice
31816
prox vertice
31817
prox vertice
31818
prox vertice
31819
prox vertice
31820
prox vertice
31821
prox vertice
31822
prox vertice
31823
prox vertice
31824
prox vertice
31825
prox vertice
31826
prox vertice
31827
prox vertice
31828
prox vertice
31829
prox vertice
31830
prox vertice

prox vertice
32257
prox vertice
32258
prox vertice
32259
prox vertice
32260
prox vertice
32261
prox vertice
32262
prox vertice
32263
prox vertice
32264
prox vertice
32265
prox vertice
32266
prox vertice
32267
prox vertice
32268
prox vertice
32269
prox vertice
32270
prox vertice
32271
prox vertice
32272
prox vertice
32273
prox vertice
32274
prox vertice
32275
prox vertice
32276
prox vertice
32277
prox vertice
32278
prox vertice
32279
prox vertice
32280
prox vertice
32281
prox vertice
32282
prox vertice
32283
prox vertice
32284
prox vertice
32285
prox vertice
32286
prox vertice
32287
prox vertice
32288
prox vertice
32289
prox vertice
32290
prox vertice
32291
prox vertice
32292
prox vertice
32293
prox vertice
32294
prox vertice
32295
prox vertice
32296
prox vertice
32297
prox vertice
32298
prox vertice
32299
prox vertice
32300
prox vertice
32301
prox vertice
32302
prox vertice
32303
prox vertice
32304
prox vertice
32305
prox vertice
32306
prox vertice
32307
prox vertice
32308
prox vertice

prox vertice
32735
prox vertice
32736
prox vertice
32737
prox vertice
32738
prox vertice
32739
prox vertice
32740
prox vertice
32741
prox vertice
32742
prox vertice
32743
prox vertice
32744
prox vertice
32745
prox vertice
32746
prox vertice
32747
prox vertice
32748
prox vertice
32749
prox vertice
32750
prox vertice
32751
prox vertice
32752
prox vertice
32753
prox vertice
32754
prox vertice
32755
prox vertice
32756
prox vertice
32757
prox vertice
32758
prox vertice
32759
prox vertice
32760
prox vertice
32761
prox vertice
32762
prox vertice
32763
prox vertice
32764
prox vertice
32765
prox vertice
32766
prox vertice
32767
prox vertice
32768
prox vertice
32769
prox vertice
32770
prox vertice
32771
prox vertice
32772
prox vertice
32773
prox vertice
32774
prox vertice
32775
prox vertice
32776
prox vertice
32777
prox vertice
32778
prox vertice
32779
prox vertice
32780
prox vertice
32781
prox vertice
32782
prox vertice
32783
prox vertice
32784
prox vertice
32785
prox vertice
32786
prox vertice

prox vertice
33254
prox vertice
33255
prox vertice
33256
prox vertice
33257
prox vertice
33258
prox vertice
33259
prox vertice
33260
prox vertice
33261
prox vertice
33262
prox vertice
33263
prox vertice
33264
prox vertice
33265
prox vertice
33266
prox vertice
33267
prox vertice
33268
prox vertice
33269
prox vertice
33270
prox vertice
33271
prox vertice
33272
prox vertice
33273
prox vertice
33274
prox vertice
33275
prox vertice
33276
prox vertice
33277
prox vertice
33278
prox vertice
33279
prox vertice
33280
prox vertice
33281
prox vertice
33282
prox vertice
33283
prox vertice
33284
prox vertice
33285
prox vertice
33286
prox vertice
33287
prox vertice
33288
prox vertice
33289
prox vertice
33290
prox vertice
33291
prox vertice
33292
prox vertice
33293
prox vertice
33294
prox vertice
33295
prox vertice
33296
prox vertice
33297
prox vertice
33298
prox vertice
33299
prox vertice
33300
prox vertice
33301
prox vertice
33302
prox vertice
33303
prox vertice
33304
prox vertice
33305
prox vertice

prox vertice
33738
prox vertice
33739
prox vertice
33740
prox vertice
33741
prox vertice
33742
prox vertice
33743
prox vertice
33744
prox vertice
33745
prox vertice
33746
prox vertice
33747
prox vertice
33748
prox vertice
33749
prox vertice
33750
prox vertice
33751
prox vertice
33752
prox vertice
33753
prox vertice
33754
prox vertice
33755
prox vertice
33756
prox vertice
33757
prox vertice
33758
prox vertice
33759
prox vertice
33760
prox vertice
33761
prox vertice
33762
prox vertice
33763
prox vertice
33764
prox vertice
33765
prox vertice
33766
prox vertice
33767
prox vertice
33768
prox vertice
33769
prox vertice
33770
prox vertice
33771
prox vertice
33772
prox vertice
33773
prox vertice
33774
prox vertice
33775
prox vertice
33776
prox vertice
33777
prox vertice
33778
prox vertice
33779
prox vertice
33780
prox vertice
33781
prox vertice
33782
prox vertice
33783
prox vertice
33784
prox vertice
33785
prox vertice
33786
prox vertice
33787
prox vertice
33788
prox vertice
33789
prox vertice

In [32]:
#posicionando vertices de acordo com o algoritmo SFDP, que evidencia "panelinhas"
m = draw_gt.sfdp_layout(g)
move_gt(g, m)
save_gt(g, 'projeto.net.gz')

In [33]:
#rederiza visualizacao do grafo
r = netpixi.render('projeto.net.gz')

In [77]:
#mostra os nomes dos jogadores na visualizacao
#CUIDADO QUE QUANDO VC RODA ISSO O ARQUIVO FICA INFINTAMENTE MAIS PESADO
r.vertex_set_key('name')

In [83]:
#quantidade de tags únicas
len(list_tags)

22258